# Día 2: Decoradores en Python

## Descripción General

Los decoradores son una de las características más poderosas y elegantes de Python. Permiten modificar o extender el comportamiento de funciones y métodos sin cambiar su código fuente. En este notebook, exploraremos los decoradores integrados más comunes (`@property`, `@staticmethod`, `@classmethod`) y aprenderemos a crear nuestros propios decoradores personalizados.

Los decoradores son esenciales en el desarrollo moderno de Python, especialmente en frameworks web como Flask y Django, y en el desarrollo de APIs con FastAPI.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender qué son los decoradores y cómo funcionan internamente
2. Utilizar decoradores integrados: `@property`, `@staticmethod`, y `@classmethod`
3. Crear decoradores personalizados simples y con parámetros
4. Aplicar `functools.wraps` para preservar metadatos de funciones
5. Identificar casos de uso apropiados para cada tipo de decorador

## 1. ¿Qué son los Decoradores?

### El Problema que Resuelven

Imagina que tienes múltiples funciones y necesitas agregar funcionalidad común a todas ellas, como:
- Medir el tiempo de ejecución
- Registrar llamadas en logs
- Validar permisos de acceso
- Cachear resultados

Sin decoradores, tendrías que modificar cada función individualmente o envolver cada llamada manualmente, lo que resulta en código repetitivo y difícil de mantener.

### ❌ MAL: Sin Decoradores

In [ ]:
import time

def calculate_sum(n: int) -> int:
    """Calculate sum without decorator."""
    start_time = time.time()  # Timing logic mixed with business logic
    result = sum(range(n))
    end_time = time.time()
    print(f"Execution time: {end_time - start_time:.4f}s")
    return result

def calculate_product(n: int) -> int:
    """Calculate product without decorator."""
    start_time = time.time()  # Repeated timing code
    result = 1
    for i in range(1, n + 1):
        result *= i
    end_time = time.time()
    print(f"Execution time: {end_time - start_time:.4f}s")
    return result

# Repeated timing logic in every function!

### ✅ BIEN: Con Decoradores

In [ ]:
import time
from functools import wraps

def timing_decorator(func):
    """Decorator to measure execution time."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f}s")
        return result
    return wrapper

@timing_decorator
def calculate_sum(n: int) -> int:
    """Calculate sum with decorator."""
    return sum(range(n))

@timing_decorator
def calculate_product(n: int) -> int:
    """Calculate product with decorator."""
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

# Clean separation of concerns!
print(calculate_sum(1000000))
print(calculate_product(20))

### Aprendizaje Clave

Un decorador es una función que toma otra función como argumento y devuelve una nueva función con funcionalidad extendida. La sintaxis `@decorator` es azúcar sintáctico para `func = decorator(func)`.

**Referencia oficial:** [PEP 318 - Decorators for Functions and Methods](https://www.python.org/dev/peps/pep-0318)

## 2. Decorador @property

### El Problema que Resuelve

En programación orientada a objetos, a menudo necesitamos controlar el acceso a los atributos de una clase. Queremos:
- Validar valores antes de asignarlos
- Calcular valores dinámicamente
- Mantener una interfaz limpia (sin getters/setters explícitos)

El decorador `@property` permite convertir métodos en atributos, proporcionando una sintaxis elegante para getters, setters y deleters.

### ❌ MAL: Getters y Setters Explícitos

In [ ]:
class Temperature:
    """Temperature class with explicit getters/setters."""
    
    def __init__(self, celsius: float):
        self._celsius = celsius
    
    def get_celsius(self) -> float:
        """Get temperature in Celsius."""
        return self._celsius
    
    def set_celsius(self, value: float) -> None:
        """Set temperature in Celsius."""
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value
    
    def get_fahrenheit(self) -> float:
        """Get temperature in Fahrenheit."""
        return self._celsius * 9/5 + 32

# Ugly syntax
temp = Temperature(25)
print(temp.get_celsius())  # Not Pythonic
temp.set_celsius(30)       # Verbose

### ✅ BIEN: Usando @property

In [ ]:
class Temperature:
    """Temperature class using @property."""
    
    def __init__(self, celsius: float):
        self._celsius = celsius
    
    @property
    def celsius(self) -> float:
        """Get temperature in Celsius."""
        return self._celsius
    
    @celsius.setter
    def celsius(self, value: float) -> None:
        """Set temperature in Celsius with validation."""
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value
    
    @property
    def fahrenheit(self) -> float:
        """Get temperature in Fahrenheit (computed property)."""
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value: float) -> None:
        """Set temperature using Fahrenheit."""
        self.celsius = (value - 32) * 5/9

# Clean, Pythonic syntax
temp = Temperature(25)
print(f"Celsius: {temp.celsius}°C")
print(f"Fahrenheit: {temp.fahrenheit}°F")

temp.celsius = 30  # Looks like attribute access
print(f"New temp: {temp.celsius}°C")

temp.fahrenheit = 86  # Set using Fahrenheit
print(f"Celsius: {temp.celsius}°C")

# Validation works
try:
    temp.celsius = -300
except ValueError as e:
    print(f"Error: {e}")

### Aprendizaje Clave

`@property` convierte un método en un atributo de solo lectura. Usa `@property_name.setter` para permitir asignación y `@property_name.deleter` para permitir eliminación. Esto proporciona encapsulación con sintaxis limpia.

**Referencia oficial:** [Python property() built-in function](https://docs.python.org/3/library/functions.html#property)

## 3. Decoradores @staticmethod y @classmethod

### El Problema que Resuelven

En las clases, no todos los métodos necesitan acceso a la instancia (`self`) o a la clase (`cls`). Necesitamos diferentes tipos de métodos:

- **Métodos de instancia**: Operan sobre instancias específicas (usan `self`)
- **Métodos de clase**: Operan sobre la clase misma (usan `cls`)
- **Métodos estáticos**: Funciones relacionadas lógicamente con la clase pero sin acceso a `self` o `cls`

Los decoradores `@staticmethod` y `@classmethod` nos permiten definir estos diferentes tipos de métodos claramente.

### Comparación: Instance, Class y Static Methods

In [ ]:
class Pizza:
    """Pizza class demonstrating different method types."""
    
    # Class variable
    menu_prices = {"margherita": 8, "pepperoni": 10, "hawaiian": 12}
    
    def __init__(self, ingredients: list[str]):
        self.ingredients = ingredients
    
    # Instance method - needs self
    def describe(self) -> str:
        """Describe this specific pizza instance."""
        return f"Pizza with {', '.join(self.ingredients)}"
    
    # Class method - needs cls
    @classmethod
    def margherita(cls) -> 'Pizza':
        """Factory method to create a Margherita pizza."""
        return cls(["mozzarella", "tomatoes", "basil"])
    
    @classmethod
    def from_name(cls, name: str) -> 'Pizza':
        """Factory method to create pizza from menu name."""
        recipes = {
            "margherita": ["mozzarella", "tomatoes", "basil"],
            "pepperoni": ["mozzarella", "pepperoni"],
            "hawaiian": ["mozzarella", "ham", "pineapple"]
        }
        return cls(recipes.get(name, []))
    
    # Static method - needs neither self nor cls
    @staticmethod
    def is_valid_ingredient(ingredient: str) -> bool:
        """Check if an ingredient name is valid."""
        valid_ingredients = [
            "mozzarella", "tomatoes", "basil", "pepperoni", 
            "ham", "pineapple", "mushrooms", "olives"
        ]
        return ingredient.lower() in valid_ingredients
    
    @staticmethod
    def calculate_cooking_time(diameter_cm: int) -> int:
        """Calculate cooking time based on pizza size."""
        return 10 + (diameter_cm // 5)

# Using instance method
pizza1 = Pizza(["mozzarella", "mushrooms"])
print(pizza1.describe())

# Using class method (factory pattern)
pizza2 = Pizza.margherita()
print(pizza2.describe())

pizza3 = Pizza.from_name("pepperoni")
print(pizza3.describe())

# Using static method (utility function)
print(f"Is 'mozzarella' valid? {Pizza.is_valid_ingredient('mozzarella')}")
print(f"Is 'chocolate' valid? {Pizza.is_valid_ingredient('chocolate')}")
print(f"Cooking time for 30cm pizza: {Pizza.calculate_cooking_time(30)} minutes")

### Aprendizaje Clave

- `@classmethod`: Recibe la clase (`cls`) como primer argumento. Útil para métodos factory y operaciones a nivel de clase.
- `@staticmethod`: No recibe `self` ni `cls`. Útil para funciones de utilidad relacionadas con la clase.
- Ambos pueden llamarse desde la clase o desde instancias.

**Referencia oficial:** [Python classmethod() and staticmethod()](https://docs.python.org/3/library/functions.html#classmethod)

## 4. Creando Decoradores Personalizados

### Decorador Simple

Un decorador personalizado es simplemente una función que toma una función y devuelve una función modificada.

In [ ]:
def debug_decorator(func):
    """Decorator that prints function calls and results."""
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result!r}")
        return result
    return wrapper

@debug_decorator
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

@debug_decorator
def greet(name: str, greeting: str = "Hello") -> str:
    """Greet someone."""
    return f"{greeting}, {name}!"

result1 = add(5, 3)
result2 = greet("Alice", greeting="Hi")

### El Problema con Decoradores Simples

Los decoradores simples pierden los metadatos de la función original (nombre, docstring, etc.):

In [ ]:
# Without functools.wraps
def bad_decorator(func):
    """Decorator without wraps."""
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@bad_decorator
def my_function():
    """This is my function's docstring."""
    pass

print(f"Function name: {my_function.__name__}")  # 'wrapper' instead of 'my_function'
print(f"Docstring: {my_function.__doc__}")       # None instead of actual docstring

### ✅ Solución: functools.wraps

`functools.wraps` preserva los metadatos de la función original:

In [ ]:
from functools import wraps

def good_decorator(func):
    """Decorator with wraps."""
    @wraps(func)  # Preserves metadata
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@good_decorator
def my_function():
    """This is my function's docstring."""
    pass

print(f"Function name: {my_function.__name__}")  # 'my_function' ✓
print(f"Docstring: {my_function.__doc__}")       # Actual docstring ✓

### Aprendizaje Clave

Siempre usa `@wraps(func)` de `functools` en tus decoradores personalizados para preservar el nombre, docstring y otros metadatos de la función original. Esto es crucial para debugging y documentación.

**Referencia oficial:** [functools.wraps](https://docs.python.org/3/library/functools.html#functools.wraps)

## 5. Decoradores con Parámetros

### El Problema

A veces necesitamos configurar el comportamiento del decorador. Por ejemplo, un decorador de retry que intente N veces, o un decorador de cache con tiempo de expiración configurable.

Para crear decoradores con parámetros, necesitamos una función adicional que devuelva el decorador.

In [ ]:
from functools import wraps
import time

def retry(max_attempts: int = 3, delay: float = 1.0):
    """Decorator that retries a function on failure.
    
    :param max_attempts: Maximum number of retry attempts
    :type max_attempts: int
    :param delay: Delay between retries in seconds
    :type delay: float
    :return: Decorated function
    :rtype: callable
    """
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_attempts:
                        print(f"Failed after {max_attempts} attempts")
                        raise
                    print(f"Attempt {attempt} failed: {e}. Retrying in {delay}s...")
                    time.sleep(delay)
        return wrapper
    return decorator

# Using the decorator with parameters
@retry(max_attempts=3, delay=0.5)
def unreliable_function(success_rate: float = 0.3):
    """Function that randomly fails."""
    import random
    if random.random() > success_rate:
        raise ValueError("Random failure!")
    return "Success!"

# Try it (may fail or succeed after retries)
try:
    result = unreliable_function(success_rate=0.7)
    print(result)
except ValueError:
    print("Function failed even after retries")

### Otro Ejemplo: Decorador de Cache con Tiempo de Expiración

In [ ]:
from functools import wraps
import time

def timed_cache(expiration_seconds: int = 60):
    """Cache decorator with expiration time.
    
    :param expiration_seconds: Cache expiration time in seconds
    :type expiration_seconds: int
    :return: Decorated function
    :rtype: callable
    """
    def decorator(func):
        cache = {}
        
        @wraps(func)
        def wrapper(*args):
            current_time = time.time()
            
            # Check if result is in cache and not expired
            if args in cache:
                result, timestamp = cache[args]
                if current_time - timestamp < expiration_seconds:
                    print(f"Cache hit for {args}")
                    return result
            
            # Compute and cache result
            print(f"Computing result for {args}")
            result = func(*args)
            cache[args] = (result, current_time)
            return result
        
        return wrapper
    return decorator

@timed_cache(expiration_seconds=5)
def expensive_computation(n: int) -> int:
    """Simulate expensive computation."""
    time.sleep(1)  # Simulate work
    return n * n

# First call - computes
print(expensive_computation(10))

# Second call - uses cache
print(expensive_computation(10))

# Different argument - computes
print(expensive_computation(20))

### Aprendizaje Clave

Los decoradores con parámetros requieren tres niveles de funciones: la función externa recibe los parámetros, devuelve el decorador, que a su vez devuelve el wrapper. Estructura: `parametros -> decorador -> wrapper -> función original`.

**Referencia oficial:** [PEP 318 - Decorators for Functions and Methods](https://www.python.org/dev/peps/pep-0318)

## Ejercicios Prácticos

### Ejercicio 1: Clase con @property (Básico)

Crea una clase `BankAccount` con:
- Un atributo privado `_balance`
- Una propiedad `balance` (solo lectura)
- Métodos `deposit()` y `withdraw()` que modifiquen el balance
- Validación: el balance nunca puede ser negativo

In [ ]:
# Tu código aquí
class BankAccount:
    pass

# Prueba tu código
account = BankAccount(100)
print(account.balance)  # Should print 100
account.deposit(50)
print(account.balance)  # Should print 150
account.withdraw(30)
print(account.balance)  # Should print 120

### Ejercicio 2: @classmethod Factory (Intermedio)

Crea una clase `Date` con:
- Atributos: `day`, `month`, `year`
- Un método de clase `from_string()` que cree una instancia desde un string "DD-MM-YYYY"
- Un método de clase `today()` que cree una instancia con la fecha actual
- Un método `display()` que muestre la fecha en formato legible

In [ ]:
# Tu código aquí
from datetime import datetime

class Date:
    pass

# Prueba tu código
date1 = Date.from_string("25-12-2024")
print(date1.display())  # Should print something like "25 de diciembre de 2024"

date2 = Date.today()
print(date2.display())  # Should print today's date

### Ejercicio 3: Decorador Personalizado (Avanzado)

Crea un decorador `@validate_types` que:
- Verifique que los argumentos de una función coincidan con sus type hints
- Lance un `TypeError` si los tipos no coinciden
- Use `functools.wraps` para preservar metadatos

Pista: Usa `func.__annotations__` para obtener los type hints.

In [ ]:
# Tu código aquí
from functools import wraps

def validate_types(func):
    pass

# Prueba tu código
@validate_types
def add_numbers(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

print(add_numbers(5, 3))  # Should work

try:
    print(add_numbers("5", 3))  # Should raise TypeError
except TypeError as e:
    print(f"Error caught: {e}")

## Resumen

En este notebook hemos aprendido:

1. **Decoradores básicos**: Funciones que modifican otras funciones, proporcionando una forma elegante de extender funcionalidad

2. **@property**: Convierte métodos en atributos, permitiendo getters/setters con sintaxis limpia y validación encapsulada

3. **@staticmethod y @classmethod**: Definen diferentes tipos de métodos en clases - estáticos para utilidades y de clase para factories

4. **functools.wraps**: Preserva metadatos de funciones decoradas, esencial para debugging y documentación

5. **Decoradores con parámetros**: Permiten configurar el comportamiento del decorador mediante una capa adicional de funciones

Los decoradores son una herramienta fundamental en Python moderno, especialmente en frameworks web y desarrollo de APIs. Dominarlos te permitirá escribir código más limpio, reutilizable y mantenible.

## Preguntas de Autoevaluación

### 1. ¿Qué hace realmente la sintaxis `@decorator` cuando se coloca sobre una función?

**Respuesta:** La sintaxis `@decorator` es azúcar sintáctico que equivale a `func = decorator(func)`. Toma la función definida, la pasa al decorador, y reemplaza la función original con el resultado devuelto por el decorador.

### 2. ¿Cuál es la diferencia principal entre `@staticmethod` y `@classmethod`?

**Respuesta:** `@staticmethod` define un método que no recibe ni `self` ni `cls` - es una función regular dentro de la clase. `@classmethod` recibe la clase (`cls`) como primer argumento, permitiendo acceso a atributos de clase y creación de instancias (patrón factory).

### 3. ¿Por qué es importante usar `@wraps(func)` de functools en decoradores personalizados?

**Respuesta:** `@wraps(func)` preserva los metadatos de la función original (nombre, docstring, anotaciones, etc.). Sin él, la función decorada tendría el nombre y docstring del wrapper, dificultando el debugging y rompiendo herramientas de documentación.

### 4. ¿Cuándo usarías `@property` en lugar de un atributo público simple?

**Respuesta:** Usa `@property` cuando necesites: (1) validar valores antes de asignarlos, (2) calcular valores dinámicamente, (3) mantener compatibilidad al cambiar de atributo simple a propiedad computada, o (4) proporcionar atributos de solo lectura.

### 5. ¿Cómo se estructura un decorador con parámetros?

**Respuesta:** Un decorador con parámetros requiere tres niveles de funciones: (1) función externa que recibe parámetros, (2) función decorador que recibe la función a decorar, (3) función wrapper que ejecuta la lógica. Estructura: `def outer(params): def decorator(func): def wrapper(*args): ... return wrapper return decorator`.

### 6. ¿Puedes aplicar múltiples decoradores a una misma función? ¿En qué orden se aplican?

**Respuesta:** Sí, puedes apilar decoradores. Se aplican de abajo hacia arriba (el más cercano a la función se aplica primero). Por ejemplo, `@dec1 @dec2 def func()` equivale a `func = dec1(dec2(func))`.

### 7. ¿Qué ventaja tiene usar `@classmethod` para crear métodos factory en lugar de funciones independientes?

**Respuesta:** Los métodos de clase mantienen la lógica de creación dentro de la clase, funcionan correctamente con herencia (usan `cls` en lugar de hardcodear el nombre de la clase), y proporcionan una API más limpia y orientada a objetos.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[Python Decorators](https://docs.python.org/3/glossary.html#term-decorator)**: https://docs.python.org/3/glossary.html#term-decorator
  - Definición oficial de decoradores en el glosario de Python

- **[property() built-in](https://docs.python.org/3/library/functions.html#property)**: https://docs.python.org/3/library/functions.html#property
  - Documentación completa del decorador @property

- **[classmethod() and staticmethod()](https://docs.python.org/3/library/functions.html#classmethod)**: https://docs.python.org/3/library/functions.html#classmethod
  - Documentación de decoradores de métodos de clase

- **[functools.wraps](https://docs.python.org/3/library/functools.html#functools.wraps)**: https://docs.python.org/3/library/functools.html#functools.wraps
  - Documentación del decorador wraps para preservar metadatos

### Estándares/PEPs
- **[PEP 318 - Decorators for Functions and Methods](https://www.python.org/dev/peps/pep-0318)**: https://www.python.org/dev/peps/pep-0318
  - PEP que introdujo la sintaxis de decoradores en Python 2.4

- **[PEP 3129 - Class Decorators](https://www.python.org/dev/peps/pep-3129)**: https://www.python.org/dev/peps/pep-3129
  - PEP que extendió decoradores a clases en Python 2.6

### Herramientas Relacionadas
- **[functools module](https://docs.python.org/3/library/functools.html)**: https://docs.python.org/3/library/functools.html
  - Módulo con herramientas para trabajar con funciones de orden superior

### Mejores Prácticas
- **[Python Wiki - PythonDecorators](https://wiki.python.org/moin/PythonDecorators)**: https://wiki.python.org/moin/PythonDecorators
  - Guía comunitaria sobre decoradores con ejemplos y patrones

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente
- Los decoradores son ampliamente usados en frameworks como Flask, Django, y FastAPI